In [1]:
from numpy.linalg import inv, eig
from numpy.linalg import matrix_rank as rank
from numpy import array, eye, dot, zeros, poly, ones
from numpy import roots, arange, asarray, concatenate
from functools import reduce
import matplotlib.pyplot as plt
from control import acker, place, ss, forced_response
from control import obsv, ctrb, minreal, tf, ss2tf, tf2ss
from IPython.display import Image

def mul_mat(*args):
    return reduce(dot, args)

def plota_resp(T,
               yout,
               titulo,
               xlabel='Tempos (s)',
               ylabel='Altura da bolinha (m)\n'):

    plt.plot(T, yout)
    plt.title(titulo + '\n')
    plt.xlabel('\n' + xlabel)
    plt.ylabel(ylabel + '\n')
    plt.grid(alpha=.5)
    plt.show()
    
def plota_resp_sist(t, yout):
    
    plt.rcParams["figure.figsize"] = (11,7)
    
    fig, axes = plt.subplots(4, 1)
    ylabels = ["x1", "x1'", "x2", "x2'"]

    for num, ax in enumerate(axes.flatten()):
        ax.axhline(y=0, color='black')
        ax.plot(T, yout[num], color='b')
        ax.set_ylabel(ylabels[num])
        ax.grid()

    fig.tight_layout()
    plt.show()    
    
# plots config    
plt.rcParams["figure.figsize"] = (11,7)
plt.rcParams.update({'font.size': 15})


In [2]:

# numerador e denominador da função transferência
n = [2, 3, -4]
d = [1, -1, 3, -1]

out = tf2ss(n, d)

In [3]:
out.A.round()

matrix([[ 1.,  3., -1.],
        [-1., -0.,  0.],
        [ 0.,  1.,  0.]])

In [4]:
out.B.round()

matrix([[-1.],
        [ 0.],
        [ 0.]])

In [5]:
out.C.round()

matrix([[-2.,  3., -4.]])

In [6]:
out.D.round()

matrix([[0.]])

$$ $$
# P1
$$ $$

$$ $$
# Controlador 
$$ $$

In [7]:
# Modelo de planta utilizado
A = array([[4,  -2],
           [15, -9]])

B = array([[1, 1]]).T

C = array([1, 0])
C2 = array([[1, 0]])

D = array([0])

out = ss2tf(A,B,C,D)

sys = ss (A,B,C,D)

In [8]:
out

TransferFunction(array([1., 7.]), array([ 1.,  5., -6.]))

In [9]:
# [Ac,Bc,Cc,T] = ctrbf(A,B,C)
# [Ao,Bo,Co,T] = obsvf(A,B,C)

In [10]:
P = array([B, dot(A,B)])

In [11]:
P = P.reshape((2,2)).T

P

array([[1, 2],
       [1, 6]])

In [12]:
T = P

Abar = mul_mat(inv(T), A, T)
Bbar = dot(inv(T), B)
Cbar = dot(C, T)

In [13]:
iP = inv(P)

iP

array([[ 1.5 , -0.5 ],
       [-0.25,  0.25]])

In [14]:
iP[1,:]

array([-0.25,  0.25])

In [15]:
p = iP[1,:]

iPP=[p, dot(p,A)]

T = inv(iPP)

Ac = mul_mat(inv(T), A, T)

Bc = dot(inv(T), B)

Cc = dot(C, T)

In [16]:
Ac

array([[ 0.,  1.],
       [ 6., -5.]])

In [17]:
Bc

array([[0.],
       [1.]])

In [18]:
Cc

array([7., 1.])

In [19]:
Kc = array([8, -2])

K = dot(Kc,inv(T))

K = array([K])

K

array([[-7.5,  5.5]])

In [20]:
D, V = eig(A-dot(B,K))
D

array([-1., -2.])

In [21]:
# checar pela F. de Ackerman
en = array([0, 1])
phi_des = dot(A,A) + 3*A + 2*eye(2)

phi_des

array([[  0.,   4.],
       [-30.,  26.]])

In [22]:
K2 = mul_mat(en, inv(P), phi_des)
K2

array([-7.5,  5.5])

In [23]:
inv(P)

array([[ 1.5 , -0.5 ],
       [-0.25,  0.25]])

In [24]:
K==K2

array([[ True,  True]])

$$ $$
# Observador 
$$ $$

In [25]:
Q = array([C, dot(C,A)])

T = inv(Q)

Abar = mul_mat(inv(T), A, T)
Bbar = dot(inv(T), B)
Cbar = dot(C, T)

In [26]:
Q

array([[ 1,  0],
       [ 4, -2]])

In [27]:
rank(Q)

2

In [28]:
iQ = inv(Q)

In [29]:
iQ

array([[ 1. ,  0. ],
       [ 2. , -0.5]])

In [30]:
iQ[:,1]

array([ 0. , -0.5])

In [31]:
q = iQ[:,1]

QQ = array([q, dot(A,q)]).T

T = (QQ)

Ao = mul_mat(inv(T), A, T)

Bo = dot(inv(T), B)

Co = dot(C, T)

In [32]:
#T.reshape((2,2))
T

array([[ 0. ,  1. ],
       [-0.5,  4.5]])

In [33]:
Ao

array([[ 0.,  6.],
       [ 1., -5.]])

In [34]:
Bo

array([[7.],
       [1.]])

In [35]:
Co

array([0., 1.])

In [36]:
Lo = array([[54, 9]]).T

L = dot(T,Lo)

L

array([[ 9. ],
       [13.5]])

In [37]:
D, V = eig(A-dot(L,C2))
D

array([-6., -8.])

In [38]:
# checar pela F. de Ackerman
Pbar = array([C.T, dot(A.T,C.T)]).T

Pbar

array([[ 1,  4],
       [ 0, -2]])

In [39]:

en = array([0, 1])
phi_des = dot(A.T,A.T) + 14*A.T + 48*eye(2)

phi_des

array([[ 90., 135.],
       [-18., -27.]])

In [40]:
L = mul_mat(en, inv(Pbar), phi_des)

L

array([ 9. , 13.5])

In [41]:
D2 = array([[0]])

a1_ = concatenate((A, B), axis=1)
a2_ = concatenate((C2, D2), axis=1)

AA = concatenate((a1_, a2_), axis=0)


In [42]:
A

array([[ 4, -2],
       [15, -9]])

In [43]:
B

array([[1],
       [1]])

In [44]:
C2

array([[1, 0]])

In [45]:
D2

array([[0]])

In [46]:
AA

array([[ 4, -2,  1],
       [15, -9,  1],
       [ 1,  0,  0]])

In [47]:
# set point é 2
rss = 2
t = array([[0, 0, rss]]).T

t

array([[0],
       [0],
       [2]])

In [48]:
# xss -> 2x1
# uss -> 1x1
# t = [[xss], [uss]]
aux = dot(inv(AA),t)


xss = aux[0:2,:]

# valor de controle
uss = aux[2,:]

In [56]:
aux

array([[ 2.        ],
       [ 3.14285714],
       [-1.71428571]])

In [49]:
# x1 tem que ficar em 2 e x2 ficar em 3.14
xss

array([[2.        ],
       [3.14285714]])

In [55]:
uss

array([-1.71428571])

In [50]:
K

array([[-7.5,  5.5]])

In [51]:
Nx = xss/rss
Nu = uss/rss
Nbar = Nu + dot(K,Nx)

In [52]:
Nbar = float(Nbar)

In [53]:
Nbar

0.2857142857142855

In [54]:

# construindo At
a11 = A - dot(B,K)
a12 = dot(B,K)
a21 = zeros(A.shape)
a22 = A - dot(L,C2)

a1_ = concatenate((a11, a12), axis=1)
a2_ = concatenate((a21, a22), axis=1)

Ace = concatenate((a1_, a2_), axis=0)

# construindo Bt
b11 = B*Nbar
b21 = zeros(B.shape)

Bce = concatenate((b11, b21), axis=0)

# construindo Ct
c11 = C2
c12 = zeros(C2.shape)

Cce = concatenate((c11, c12), axis=1)


ValueError: shapes (2,) and (1,2) not aligned: 2 (dim 0) != 1 (dim 0)

In [ ]:
Ace

In [ ]:
Bce

In [ ]:
Cce